In [24]:
import copy
from aerosandbox import *
from aerosandbox.library.airfoils import e216, naca0008
import time
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

opti = cas.Opti()  # Initialize an analysis/optimization environment

In [25]:
## define constants
v = 15 
aoa = 5

In [26]:
# Define Geometry
# Here, all distances are in meters and all angles are in degrees.
airplane = Airplane(
    name="RollingRigSimple",
    x_ref=0,  # CG location
    y_ref=0,  # CG location
    z_ref=0,  # CG location
    wings=[
        Wing(
            name="Main Wing",
            x_le=0,  # Coordinates of the wing's leading edge
            y_le=0,  # Coordinates of the wing's leading edge
            z_le=0,  # Coordinates of the wing's leading edge
            symmetric=True,
            xsecs=[  # The wing's cross ("X") sections
                WingXSec(  # left tip
                    x_le=0,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    y_le=0,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    z_le=0,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=0.067,
                    twist=0,  # degrees
                    airfoil=naca0008,  # Airfoils are blended between a given XSec and the next one.
                    spanwise_panels = 30,
                ),
                WingXSec(  # left tip
                    x_le=0,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    y_le=0.5,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    z_le=0,  # Coordinates of the XSec's leading edge, relative to the wing's leading edge.
                    chord=0.067,
                    twist=0,  # degrees
                    airfoil=naca0008,  # Airfoils are blended between a given XSec and the next one.
                ),
            ]
        ),
    ]
)

In [27]:
ap = Casll1(  # Set up the AeroProblem
    airplane=airplane,
    op_point=OperatingPoint(
        density=1.225,  # kg/m^3
        viscosity=1.81e-5,  # kg/m-s
        velocity=v,  # m/s
        mach=0,  # Freestream mach number
        alpha=aoa,  # In degrees
        beta=0,  # In degrees
        p=np.deg2rad(0),  # About the body x-axis, in rad/sec
        q=0,  # About the body y-axis, in rad/sec
        r=0,  # About the body z-axis, in rad/sec
    ),
    opti=opti,  # Pass it an optimization environment to work in
    run_setup=False
)
ap.setup(verbose=False)
# Solver options
opti.solver('ipopt')

In [28]:
# Solve
start_time = time.time()
sol = opti.solve()
print("--- %s seconds ---" % (time.time() - start_time))
# left_fwt_vert_L =np.sum(ap_sol.) 
# Postprocess

# Create solved object
ap_sol = copy.deepcopy(ap)
ap_sol.substitute_solution(sol)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      900
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      465

Total number of variables............................:       30
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       30
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [29]:
# Plot Geometry
ap_sol.draw()

In [30]:
# Generate Lift Distrobution
x = list([i[1] for i in ap_sol.vortex_centers])
y = list([np.sqrt(x[1]**2 + x[2]**2) for x in 2*ap_sol.forces_inviscid_geometry]/(v**2*ap_sol.areas*1.225*np.deg2rad(aoa)))
x.append(0.5)
y.append(0)

In [33]:
# Plot Lift Distributuion
fig = px.scatter(x=x, y=y)
fig.add_trace(go.Line(x=[0.5*0.782]*2,y=[0,2*np.pi]))
fig.add_trace(go.Line(x=[0,0.5*0.782,0.5]*2,y=[2*np.pi,2*np.pi,2*np.pi]))
fig.add_trace(go.Line(x=[0,0.5*0.782,0.5]*2,y=[2*np.pi,2*np.pi,0]))
fig.show()

In [37]:
# save Lift Distrobution
record = [{"y":x_i,"C_l":y[i]} for i,x_i in enumerate(x)]

df = pd.DataFrame(record)
df.to_csv(r'LiftDistribution.csv', index = False)